In [57]:
import future, sys, os, datetime, argparse
# print(os.path.dirname(sys.executable))
import torch
import numpy as np
import matplotlib
from tqdm import tqdm
# from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
matplotlib.rcParams["figure.figsize"] = [10, 10]

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


import wandb
api = wandb.Api()

def load_dataframe_wandb(runs):
    # Change oreilly-class/cifar to <entity/project-name>
    runs = api.runs(runs)
    summary_list = []
    config_list = []
    name_list = []
    for run in runs:
    	# run.summary are the output key/values like accuracy.  We call ._json_dict to omit large files
    	summary_list.append(run.summary._json_dict)

    	# run.config is the input metrics.  We remove special values that start with _.
    	config_list.append({k: v for k, v in run.config.items() if not k.startswith('_')})

    	# run.name is the name of the run.
    	name_list.append(run.name)

    import pandas as pd

    summary_df = pd.DataFrame.from_records(summary_list)
    config_df = pd.DataFrame.from_records(config_list)
    name_df = pd.DataFrame({'name': name_list})
    all_df = pd.concat([name_df, config_df, summary_df], axis=1)

    return all_df

In [58]:

data = load_dataframe_wandb("mlmd/BiDirectional_MD")


In [122]:
data_set = 'keto_1fs.npz'

for output_length in [2, 5, 10, 20]:

    for pct_data_set in [0.001, 0.01, 0.1, 0.25, 0.5, 1.0]:
        s = ""
        for model in ['ode', 'hamiltonian', 'rnn', 'lstm', 'bi_ode', 'bi_hamiltonian', 'bi_rnn', 'bi_lstm']:
            tmp = data[(data.data_set==data_set)&(data.output_length==output_length)&(data.pct_data_set==pct_data_set)&(data.model==model)]
            try:
                if model!='bi_lstm': s += f"{tmp['val/mse'].to_list()[0]:.3f}"+"\t \t & "
                if model=='bi_lstm': s += f"{tmp['val/mse'].to_list()[0]:.3f}"+"\\\ "
            except:
                if model!='bi_lstm': s += "-.--- \t \t & "
                if model=='bi_lstm': s += "-.--- \\\ "
            # print(tmp['val/mse'].to_list())
        print(s)
    print('')

0.082	 	 & 0.060	 	 & 0.063	 	 & 0.061	 	 & 0.004	 	 & 0.004	 	 & 0.004	 	 & 0.004\\ 
0.063	 	 & 0.060	 	 & 0.065	 	 & 0.060	 	 & 0.004	 	 & 0.004	 	 & 0.004	 	 & 0.004\\ 
0.067	 	 & 0.059	 	 & 0.056	 	 & 0.064	 	 & 0.004	 	 & 0.004	 	 & 0.003	 	 & 0.004\\ 
0.063	 	 & 0.053	 	 & 0.048	 	 & 0.062	 	 & 0.002	 	 & 0.002	 	 & 0.001	 	 & 0.002\\ 
0.068	 	 & 0.057	 	 & 0.051	 	 & 0.066	 	 & -.--- 	 	 & -.--- 	 	 & 0.001	 	 & 0.001\\ 
0.037	 	 & 0.030	 	 & 0.043	 	 & 0.032	 	 & -.--- 	 	 & -.--- 	 	 & 0.000	 	 & 0.000\\ 

0.446	 	 & 0.252	 	 & 0.275	 	 & 0.255	 	 & 0.054	 	 & 0.053	 	 & 0.056	 	 & 0.053\\ 
0.263	 	 & 0.252	 	 & 0.264	 	 & 0.246	 	 & 0.053	 	 & 0.053	 	 & 0.055	 	 & 0.053\\ 
0.224	 	 & 0.233	 	 & 0.178	 	 & 0.211	 	 & 0.058	 	 & 0.051	 	 & 0.043	 	 & 0.063\\ 
0.210	 	 & 0.192	 	 & 0.149	 	 & 0.204	 	 & 0.056	 	 & 0.021	 	 & 0.008	 	 & 0.057\\ 
0.212	 	 & 0.211	 	 & 0.159	 	 & 0.211	 	 & -.--- 	 	 & -.--- 	 	 & 0.003	 	 & 0.017\\ 
0.112	 	 & 0.095	 	 & 0.096	 	 & 0.090	 	 & -.-